In [1]:
import os
import pandas as pd
import numpy as np
import re
import string
import json
import zipfile
import matplotlib.pyplot as plt
from os.path import isfile, join
from random import sample 
import sqlalchemy as sa
import ast
from json.decoder import JSONDecodeError
import seaborn as sns
from functools import reduce
import warnings
import tqdm
from tqdm.notebook import tqdm, trange
import time
import pykakasi
import sqlite3
from thefuzz import process
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm, trange

/Users/yuikondo/opt/anaconda3/lib/python3.9/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Load Data

In [16]:
gender_df = pd.read_csv("data/processed/learning_data.csv")
gender_df

,Unnamed: 0,username,user_full_name,gender,user_bio
0,1,ipppen,尊厳死 a.k.a.ipppen,NaN,小児リアリスト達に捧ぐ第三次サマーオブラヴ
1,2,bessei_family,選択的夫婦別姓制度の実現へ,NaN,現在日本で言われている〝夫婦別姓〟とは「選択的夫婦別姓」であり、「強制的夫婦別姓」ではありま...
2,3,hiro_npwr,まさむね,NaN,NaN
3,4,subaru1000,47×17,NaN,love all kinds of aircrafts include fixed gear...
4,5,mumumu_fz,いろむ,NaN,考えることを面倒くさがらずにいたい。
...,...,...,...,...,...
3217,3994,atsusugiru___,暑すぎ無理,NaN,目についたもの気になったものなんでも。よく無言フォローします。嫌な人はブロックしてね。
3218,3995,naya1178,ナヤ,NaN,50代シュフです\nコロナ禍で家族の命と健康を守るための情報収集アカウントになりました\nど...
3219,3997,AXzFptbugHjCtVf,kei,NaN,すでにテレビ局は親韓や親中の反日工作機関ばかり。テレビ局は中国や韓国に乗っ取られている。移民...
3220,3998,mitarasi_cat,みたらしだんご,NaN,日本大好き。フィギュアスケート大好き。ネコも好き。日本を好きでいてくれる国を大事にしたい＆理...


## Null Values

In [17]:
# Check Null values
gender_df.isna().sum()

# Drop values
gender_df.dropna(axis=0,inplace=True)

In [18]:
gender_df

,Unnamed: 0,username,user_full_name,gender,user_bio
5,6,nakani_kazuto,中西一登,1.0,SE→退職→山梨で友達と畑→結婚→新宿に戻り夫婦で２年のんびり→2011年９月から京都の京丹...
8,9,takapee100,たかびー🇺🇦☂☀️,0.0,♀
12,13,bamunomama,バムのママ,0.0,ちかこ
14,15,ogntom,桜音,0.0,猫好きです。他の情報はツイートなどの履歴を見て下さい。\n\nKeywords: 論理、数学...
18,19,highflyflo,kAz(プロレス・政治・世界情勢その他),1.0,最近ちょっと政治や社会情勢のtweetやリプが増えてきてしまっているのので、名前変えました。...
...,...,...,...,...,...
2822,3500,ban3yyz,Lucy.T.Itagaki,0.0,日本学術会議問題からTwitter始めました。非正規の大学教員、専門は中国語😘自由と人権と香...
2825,3504,mohikan71,三井嘉雄,1.0,田舎の弁護士です
2836,3518,ethan_thx_hunt,イーサン・ハント4世💙💛 Ethan Hunt Ⅳ,0.0,ずっとアラフォーだと思ってたのにいつの間にかアラフィフに。 #映画 と #テレビドラマ が私...
2840,3523,NTUY_uncle_bot,ネトウヨの毒舌な伯父さんbot,1.0,いつも強がりばかり言ってる甥っ子・姪っ子たちを応援するbot。応援しているのに、なぜか反発さ...


In [19]:
# count the gender column
gender_df['gender'].value_counts()

0.0    507
1.0    493
Name: gender, dtype: int64

## Clean Data

In [21]:
# Import libraries
import re
import spacy
from matplotlib import pyplot as plt

In [79]:
# Clean text
def clean_text(text):
    # Reduce multiple spaces and newlines to only one
    text = re.sub(r'(\s\s+|\n\n+)', r'\1', text)
    # Remove double quotes
    text = re.sub(r'"', '', text)
    text = re.sub(r'、', '', text)
    text = re.sub(r'。', '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\u3000', '', text)
    text = re.sub(r'\d+\.*\d*', '0', text)
    return text

gender_df['clean_bio'] = gender_df['user_bio'].apply(clean_text)
gender_df['clean_user_full_name'] = gender_df['user_full_name'].apply(clean_text)
# Remove hyperlinks
gender_df['clean_bio'] = gender_df['clean_bio'].str.replace('http\S+|www.\S+', '', case=False)
# Remove patterns
def remove_pattern(text,pattern):
    # re.findall() finds the pattern i.e @user and puts it in a list for further task
    r = re.findall(pattern,text)
    # re.sub() removes @user from the sentences in the dataset
    for i in r:
        text = re.sub(i,"",text)
    return text

gender_df['clean_bio'] = np.vectorize(remove_pattern)(gender_df['clean_bio'], "@[\w]*") # Removes all @
gender_df['clean_bio'] = np.vectorize(remove_pattern)(gender_df['clean_bio'], "&amp;") # Removes all &

In [80]:
gender_df

,Unnamed: 0,username,user_full_name,gender,user_bio,clean_bio,clean_user_full_name
5,6,nakani_kazuto,中西一登,1.0,SE→退職→山梨で友達と畑→結婚→新宿に戻り夫婦で２年のんびり→2011年９月から京都の京丹...,SE→退職→山梨で友達と畑→結婚→新宿に戻り夫婦で0年のんびり→0年0月から京都の京丹後市に...,中西一登
8,9,takapee100,たかびー🇺🇦☂☀️,0.0,♀,♀,たかびー🇺🇦☂☀️
12,13,bamunomama,バムのママ,0.0,ちかこ,ちかこ,バムのママ
14,15,ogntom,桜音,0.0,猫好きです。他の情報はツイートなどの履歴を見て下さい。\n\nKeywords: 論理、数学...,猫好きです他の情報はツイートなどの履歴を見て下さいKeywords: 論理数学物理; 積極財...,桜音
18,19,highflyflo,kAz(プロレス・政治・世界情勢その他),1.0,最近ちょっと政治や社会情勢のtweetやリプが増えてきてしまっているのので、名前変えました。...,最近ちょっと政治や社会情勢のtweetやリプが増えてきてしまっているのので名前変えました［趣...,kAz(プロレス・政治・世界情勢その他)
...,...,...,...,...,...,...,...
2822,3500,ban3yyz,Lucy.T.Itagaki,0.0,日本学術会議問題からTwitter始めました。非正規の大学教員、専門は中国語😘自由と人権と香...,日本学術会議問題からTwitter始めました非正規の大学教員専門は中国語😘自由と人権と香港を...,Lucy.T.Itagaki
2825,3504,mohikan71,三井嘉雄,1.0,田舎の弁護士です,田舎の弁護士です,三井嘉雄
2836,3518,ethan_thx_hunt,イーサン・ハント4世💙💛 Ethan Hunt Ⅳ,0.0,ずっとアラフォーだと思ってたのにいつの間にかアラフィフに。 #映画 と #テレビドラマ が私...,ずっとアラフォーだと思ってたのにいつの間にかアラフィフに #映画 と #テレビドラマ が私の...,イーサン・ハント0世💙💛 Ethan Hunt Ⅳ
2840,3523,NTUY_uncle_bot,ネトウヨの毒舌な伯父さんbot,1.0,いつも強がりばかり言ってる甥っ子・姪っ子たちを応援するbot。応援しているのに、なぜか反発さ...,いつも強がりばかり言ってる甥っ子・姪っ子たちを応援するbot応援しているのになぜか反発される...,ネトウヨの毒舌な伯父さんbot


In [81]:
gender_df["clean_bio"].to_list()

['SE→退職→山梨で友達と畑→結婚→新宿に戻り夫婦で0年のんびり→0年0月から京都の京丹後市に移住して有機農家「梅本農場」で研修→家を買って定住興味：整体パーマカルチャーNVC畑おしゃべりシーカヤック湯たんぽ灯籠など',
 '♀',
 'ちかこ',
 '猫好きです他の情報はツイートなどの履歴を見て下さいKeywords: 論理数学物理; 積極財政反緊縮; 反陰謀論批判的思考共和主義(反王制)反差別自由・独立・民主反専制; 猫SF圏論糖質制限武装独立・反自民改憲派反自公維非右翼・非共産脱原発・再エネUBI死刑廃止医療大麻',
 '最近ちょっと政治や社会情勢のtweetやリプが増えてきてしまっているのので名前変えました［趣味:新日本プロレス観戦(TV含)です］フォローはして頂けたらなるべくお返し致します🙇\u200d♂️',
 '俳優青年団青森県十和田市出身FtM. GID.俳優・勉強会の仕事はホームページからご依頼・お問い合わせ下さい●0月舞台ムニ『ことばにない』出演0/0(木)-0(日)/0(土)-0(月) THEATRE E0 KYOTO',
 '見た目国籍不明のアヤシイ人その実は大阪のおばちゃん信仰はキリスト教プロテスタント嫌いなものはレイシズム「東西文明の十字路」という言葉に弱い🇹🇷好き(ただしアヤソフィア聖堂のモスク化は憂う)性格は穏やかです#実技試験桜組',
 'Wife-loving, corgi/Akita-adoring & NFC-serving human trying to find beauty in Japan. Ex-Zoo/Van Ness, Zamalek dweller. Usual disclaimers. Enjoy life together.',
 '君が代は千代に八千代にさざれ石の巌となりて苔のむすまで／ 私のリツイートは必ずしも賛同ではない事をここに宣言致します',
 '成人済み腐女子履修履歴/最新はアロルク0ギャクラレオ強竜赤青他にも作家と准教授のあれとシノヤマとか色々あんまり地雷はないあとイカやってます',
 '❤️トピア垢💙不定期で歌枠やってます ♯Palette LIKE→ヨルシカ/YOASOBI/傘村トータ/梶浦由記/澤野弘之/音ゲー/アニメ',
 'アイコン→紅猫りお ヘッダー→みつばちさん 🐝 (*´︶`

## Remove Stop Words (WIP)

In [82]:
# nlp = spacy.load('en')
# def convert_text(text):
#     sent = nlp(text)
#     ents = {x.text: x for x in sent.ents}
#     tokens = []
#     for w in sent:
#         if w.is_stop or w.is_punct:
#             continue
#         if w.text in ents:
#             tokens.append(w.text)
#         else:
#             tokens.append(w.lemma_.lower())
#     text = ' '.join(tokens)
# return text
# tweets['clean_text'] = tweets['clean_text'].apply(convert_text)
# tweets.sample(15)

## Build a model

In [95]:
#import libraries
import MeCab
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

text_list = gender_df["clean_bio"].to_list()
 
    
#　分かち書き
for i in range(len(text_list)):
    mecab = MeCab.Tagger ("-Owakati")
    text_list[i] = mecab.parse (text_list[i])
    
text_list[3]

bow = CountVectorizer()
count = bow.fit_transform(text_list)
vec = count.toarray()
name = bow.get_feature_names()
words_df = pd.DataFrame(vec, columns=name)

In [91]:
words_df

,0ⅱ,__,abc,above,ac,account,accounting,ace,acid,addicted,...,𓈒𓏸,𝐀𝐧𝐭𝐢𝐅𝐚𝐤𝐞𝐕𝐢𝐫𝐮𝐬𝐂𝐥𝐮𝐛,𝕓𝕖𝕒𝕦𝕥𝕚𝕗𝕦𝕝,𝕓𝕪,𝕔𝕣𝕖𝕒𝕥𝕖𝕕,𝕞𝕖𝕤𝕤,𝕞𝕚𝕤𝕗𝕚𝕥𝕤,𝙲𝙾𝚂𝙼𝙴,𝙵𝙰𝚂𝙷𝙸𝙾𝙽,𝙹𝙼𝚂𝙳𝙵
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
# Splitting the data into training and test set

X = words_df
y = gender_df['gender']

# Use Bag-of-words features
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X, y, test_size=0.20)

In [94]:
# Logistic Regression

# Fitting on Logistic Regression model
logreg = LogisticRegression()
logreg.fit(X_train_bow, y_train_bow)
prediction_bow = logreg.predict_proba(X_test_bow)
# Calculating the F1 score
# If prediction is greater than or equal to 0.5 than 1, else 0
# Gender, 0 = female and 1 = male
prediction_int = prediction_bow[:,1]>=0.5
prediction_int = prediction_int.astype(np.int)
# Calculating F1 score
log_bow = f1_score(y_test_bow, prediction_int)
log_bow

0.6041666666666666

In [ ]:
# IF Userの名前が一致すれば、それで性別判定を行う。
# If 一致しなければ、User bioを用いて機械学習を行う。